In [1]:
import pybullet as pb
import pybullet_data

import os
import numpy as np
import math
import random
import time
import csv
import PIL
#import glob 
from time import sleep
from PIL import Image
from numpy import save

In [2]:
#Start the simulation and load in table
physicsClient = pb.connect(pb.GUI, options='--background_color_red=0.2 --background_color_green=0.2 --background_color_blue=0.2S')
pb.setAdditionalSearchPath(pybullet_data.getDataPath())
planeId = pb.loadURDF('plane.urdf')
tableId = pb.loadURDF("table/table.urdf")
pb.setGravity(0, 0, -9.8)
pb.setRealTimeSimulation(1)

In [3]:
# sleep(1)
# bicycleID = pb.loadURDF("bicycle/bike.urdf", globalScaling=0.1, basePosition=[0,0,1], baseOrientation = [1,1,1,1])
# sleep(1)
# samuraiID = pb.loadURDF("samurai.urdf")
# sleep(1)
# r2d2ID = pb.loadURDF("r2d2.urdf", globalScaling=0.25, basePosition=[0,0,1], baseOrientation = [0,0,1,0])
# sleep(1)
# mugID = pb.loadURDF("objects/mug.urdf", globalScaling=1, basePosition=[0,0,1], baseOrientation = [0,0,1,0])
# sleep(1)
# racecarId = pb.loadURDF("racecar/racecar.urdf", globalScaling=1, basePosition=[0.15,0,1], baseOrientation = [0,0,1,0])
# sleep(1)
# duckId = pb.loadURDF("duck_vhacd.urdf", globalScaling=2, basePosition=[0.06,-0.05,1], baseOrientation = [0,0,1,0])
# sleep(1)
# ballID = pb.loadURDF("soccerball.urdf",globalScaling=0.1)
# sleep(1)
# teddyID = pb.loadURDF("teddy_vhacd.urdf", globalScaling=3, basePosition=[-0.05,0.1,0.45], baseOrientation = [1,0,1,1])

In [4]:
#Setup the directories where images and CSVs will be stored
current_time = time.strftime("%Y%m%d-%H%M%S")
os.makedirs("imgs", exist_ok = True)
os.chdir("imgs")
os.mkdir(current_time)
os.chdir(current_time)
img_types = ["rgbImg","segImg"] #, "depthImg", "segImg"
for t in img_types:
    os.makedirs(t, exist_ok = True)
os.chdir("../..")

In [5]:
#shapes = ['000']
#shapes = ['000', '003']
shapes = [['000',0.035],
          ['003', 0.035],
          ['006', 0.035], 
          ['008', 0.035],
          ['014', 0.035],
          ['026', 0.035]]
#shapes = ['000','006', '008', '053']#, '077']

# real_objects = [ ['bicycle/bike', 0.2, [0,0,1], [1,1,1,1]], 
#                 ['r2d2', 0.30, [0,0,1], [0,0,1,0]], 
#                 ['objects/mug',  2, [0,0,1], [0,0,1,0]], 
#                 ['racecar/racecar', 0.8, [0.15,0,1], [0,0,1,0]], 
#                 ['duck_vhacd', 3, [0.06,-0.05,1], [0,0,1,0]], 
#                 ['soccerball', 0.3, [0,0,1], [0,0,1,0]],
#                 #['teddy_vhacd', 2, [-0.05,0.1,0.45], [1,0,1,1]],
#                ]

r_range = [0.50,0.50]
obj_num = 2000
img_size = 128

# multiBodyId = pb.loadURDF(real_objects[4][0]+".urdf",
#                                   globalScaling=real_objects[4][1], 
#                                   basePosition = real_objects[4][2], 
#                                   baseOrientation = real_objects[4][3])

In [6]:
from numpy import arange, pi, sin, cos, arccos
def fibonacci_sphere():
    n = obj_num #Number of images
    goldenRatio = (1 + 5**0.5)/2
    i = arange(0, n)
    theta = 2 *pi * i / goldenRatio
    phi = arccos(1 - 2*(i+0.5)/n)
    x, y, z = cos(theta) * sin(phi), sin(theta) * sin(phi), cos(phi);
    return x,y,z,theta,phi

In [7]:
with open('imgs/'+current_time+'/rgbCSV.csv', 'w', newline='') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(['Image Name', 'Shape Type', 'Coords'])

    shapeID = 0
    depth_imgs = []
    seg_imgs = []

#############################################################################################

    #Use this!
    for shape in shapes:
        #Create the object
        visualShapeId = pb.createVisualShape(
            shapeType=pb.GEOM_MESH,
            fileName='procedural_objects/'+shape[0]+'/'+shape[0]+'.obj',
            rgbaColor=None,
            meshScale=[shape[1], shape[1], shape[1]])

        collisionShapeId = pb.createCollisionShape(
            shapeType=pb.GEOM_MESH,
            fileName='procedural_objects/'+shape[0]+'/'+shape[0]+'_coll.obj',
            meshScale=[shape[1], shape[1], shape[1]])

        multiBodyId = pb.createMultiBody(
            baseMass=1.0,
            baseCollisionShapeIndex=collisionShapeId, 
            baseVisualShapeIndex=visualShapeId,
            basePosition=[0,0,1],
            baseOrientation=pb.getQuaternionFromEuler([0, 0, 0]))

        textureId = pb.loadTexture('lead-gray.jpg')
        pb.changeVisualShape(multiBodyId, -1, textureUniqueId=textureId)

        #Allow object to fall and come to rest on table
        sleep(1)
        obj_pos, _ = pb.getBasePositionAndOrientation(multiBodyId)

#############################################################################################  

    #OR This!
#     for real_object in real_objects:
#         multiBodyId = pb.loadURDF(real_object[0]+".urdf",
#                                   textureId = pb.loadTexture('lead-gray.jpg')
#                                   globalScaling=real_object[1], 
#                                   basePosition = real_object[2], 
#                                   baseOrientation = real_object[3])
        
#############################################################################################
        sleep(4)    
        obj_pos = [0,0,0.66]


        projectionMatrix = pb.computeProjectionMatrixFOV(
            fov=45.0,
            aspect=1.0,
            nearVal=0.005,
            farVal=10)

        #Generate coords for the given radius range and no of images per obj
        r = 0.5
        x_arr,y_arr,z_arr,theta_arr,phi_arr = fibonacci_sphere()
        x_arr = x_arr+obj_pos[0]
        y_arr = y_arr+obj_pos[1]
        z_arr = z_arr+obj_pos[2]
        
        
        #should be half plus 1 of the total sphere points
        cutoff = 1000
        x_arr,y_arr,z_arr,theta_arr,phi_arr = x_arr[:cutoff],y_arr[:cutoff],z_arr[:cutoff],theta_arr[:cutoff],phi_arr[:cutoff]
        
        
        
        for i,x,y,z,theta,phi in zip(np.arange(len(x_arr)),x_arr,y_arr,z_arr,theta_arr,phi_arr):

            if phi == 0 :
                upVector =[0,1,0]
            else:
                upVector=[0,0,1]

            viewMatrix = pb.computeViewMatrix(
                cameraEyePosition=[x, y, z],
                cameraTargetPosition=obj_pos,
                cameraUpVector=upVector)

            width, height, rgbImg, depthImg, segImg = pb.getCameraImage(
                width=img_size, 
                height=img_size,
                viewMatrix=viewMatrix,
                projectionMatrix=projectionMatrix)
                
            #print(np.array(rgbImg))
            rgb_array = np.array(rgbImg, dtype=np.uint8)
            rgb_array = np.reshape(rgb_array, (height, width, 4))
            rgb_array = rgb_array[:, :, :3]

            im = Image.fromarray(rgb_array)
            im.save('imgs/'+current_time+'/rgbIMG/'+str(shapeID)+'_%04d.png' %i)

            depth_array = np.array(depthImg)
            depth_array = depth_array.reshape(height, width)
            depth_imgs.append(depth_array)

            seg_array = np.array(segImg)
            seg_array = seg_array.reshape(height, width)
            seg_imgs.append(seg_array)

            #seg_array = np.stack((seg_array,)*3, axis=-1)
            #seg_R = np.where(seg_array[:,:,0] != 2, 0, rgb_array[:,:,0])
            #seg_G = np.where(seg_array[:,:,1] != 2, 255, rgb_array[:,:,1])
            #seg_B = np.where(seg_array[:,:,2] != 2, 0, rgb_array[:,:,2])

            seg_R = np.where(seg_array != 2, 100, rgb_array[:,:,0])
            seg_G = np.where(seg_array != 2, 100, rgb_array[:,:,1])
            seg_B = np.where(seg_array != 2, 100, rgb_array[:,:,2])
            seg_rgb_array = np.stack((seg_R, seg_G, seg_B), axis=-1)

            seg_im = Image.fromarray(seg_rgb_array)
            seg_im.save('imgs/'+current_time+'/segIMG/'+str(shapeID)+'_%04d.png' %i)

            thewriter.writerow([str(shapeID)+'_%04d.png' %i, shapeID, [r,theta%(2*math.pi),phi]])


        shapeID +=1

        pb.removeBody(multiBodyId)

    depth_imgs = np.array(depth_imgs)
    save('imgs/'+current_time+'/depthImg.npy', depth_imgs)

    seg_imgs = np.array(seg_imgs)
    save('imgs/'+current_time+'/segImg.npy', seg_imgs)

In [8]:
pb.disconnect()